In [2]:
# Classes and objects
# Class : blueprint for creating objects
# Objects holds state and behavior

class Car:
    def __init__(self, brand, model):
        self.brand = brand
        self.model = model
        
    def display_info(self):
        print(f"This is a {self.brand} {self.model}.")

# Creating an object of the Car class
my_car = Car("Toyota", "Corolla")
my_car.display_info()

your_car = Car("Honda", "Civic")
your_car.display_info()

This is a Toyota Corolla.
This is a Honda Civic.


In [3]:
# Understanding class attributes and instance attributes
# Class attributes are shared across all instances of the class
# attributes are variable that belong to the class
# mthods are functions that belong to the class

class Dog:
    def __init__(self, name, breed):
        self.name = name
        self.breed = breed
        
    def bark(self):
        print(f"{self.name} is barking!")
        
dog1 = Dog("Buddy", "Golden Retriever")
dog2 = Dog("Max", "Beagle")

dog1.bark()
dog2.bark()

Buddy is barking!
Max is barking!


In [4]:
# Constructors : init method is called automatically when an object is created

class Person:
    def __init__(self, name, age):
        self.name = name
        self.age = age
        
    def greet(self):
        print(f"Hello, my name is {self.name} and I am {self.age} years old.")
        
person1 = Person("Alice", 30)
person1.greet()

Hello, my name is Alice and I am 30 years old.


In [ ]:
# Working with multiple objects
# Multiple objects can be created from the same class 

In [2]:
# Bank Account Simulator
import json

LOG_TRANSACTION = "./assets/log_transaction.json"
ACCOUNT_FILE = "./assets/accounts.json"

class BankAccount:
    def __init__(self, account_holder, initial_balance=0):
        self.account_holder = account_holder
        self.balance = initial_balance
        
    # Deposit Money
    def deposit(self, amount):
        try:
            if amount > 0:
                self.balance += amount
                print(f"Deposited {amount} €. New balance: {self.balance} €")
                update_account(self.account_holder, self.balance)
                log_transaction(self.account_holder, "Deposit", amount, self.balance)
            else:
                print("Invalid deposit amount. Amount must be greater than 0.")
        except ValueError:
            print("Incorrect value for amount to deposit.")
            
    # Withdraw Money
    def withdraw(self, amount):
        try:
            if amount > 0 and amount <= self.balance:
                self.balance -= amount
                print(f"Withdrew {amount} €. New balance: {self.balance} €")
                update_account(self.account_holder, self.balance)
                log_transaction(self.account_holder, "Withdraw", -amount, self.balance)
            else:
                print("Invalid withdrawal amount or insufficient funds.")
        except ValueError:
            print("Incorrect value for amount to withdraw.")
            
    def transfer(self, amount, recipient):
        try:
            if amount > 0 and amount <= self.balance:
                self.balance -= amount
                recipient.balance += amount
                print(f"Transferred {amount} € to {recipient.account_holder}")
                update_account(self.account_holder, self.balance)
                update_account(recipient.account_holder, recipient.balance)
                log_transaction(self.account_holder, "Transfer", -amount, self.balance, recipient)
            else:
                print("Invalid withdrawal amount or insufficient funds.")
        except ValueError:
            print("Incorrect value for amount to transfer.")
            
    # Show Account Details
    def show_details(self):
        print("\n-------- Account Details --------")
        print(f"Account Holder: {self.account_holder}")
        print(f"Account Balance: {self.balance} €")
        
# Main Program
def load_log_content():
    try:
        with open(LOG_TRANSACTION, 'r') as json_file:
            log_content = json.load(json_file)
        if log_content:
            return log_content
    except FileNotFoundError:
        print("Log transaction file not found.")
    except Exception as e:
        print(f"An unexpected error occured: {e}")
    return []

def log_transaction(holder, operation, amount, balance, recipient=None):
    try:
        log_content = load_log_content()
        transaction_data = {
            "name": holder,
            "transaction": {
                "operation": operation,
                "amount": amount,
                "balance": balance,
            }
        }
        if recipient:
            transaction_data["transaction"]["recipient_name"] = recipient.account_holder
        log_content.append(transaction_data)
        with open(LOG_TRANSACTION, 'w') as json_file:
            json.dump(log_content, json_file, indent=2)
    except Exception as e:
        print(f"An unexpected error occured: {e}")

def load_accounts():
    try:
        with open(ACCOUNT_FILE, 'r') as file:
            accounts_data = json.load(file)
        return accounts_data
    except FileNotFoundError:
        print("No account found. Please create an account first.")
    except Exception as e:
        print(f"An unexpected error occured: {e}")
    return []

def save_account(account):
    try:
        accounts = load_accounts()
        accounts.append(
            {
                "account_holder": account.account_holder,
                "balance": account.balance
            }
        )
        with open(ACCOUNT_FILE, 'w') as json_file:
            json.dump(accounts, json_file, indent=2)
    except Exception as e:
        print(f"An unexpected error occured: {e}")

def get_formatted_account():
    try:
        accounts = {}
        accounts_data = load_accounts()
        if accounts_data:
            for account in accounts_data:
                accounts[account["account_holder"]] = BankAccount(account["account_holder"], account["balance"])   
            return accounts
    except FileNotFoundError:
        print("No accounts found. Please create an account first.")
    except Exception as e:
        print(f"An unexpected error occured: {e}")
    return {}

accounts = get_formatted_account()
    
def create_account():
    name = input("Enter account holder's name: ").strip()
    initial_deposit = float(input("Enter initial balance: "))
    account = BankAccount(name, initial_deposit)
    accounts[name] = account
    save_account(account)
    print("Account created successfully!")
    log_transaction(name, "Create", initial_deposit, initial_deposit)
    
def update_account(account_holder, balance):
    try:
        accounts = load_accounts()
        for account in accounts:
            if account["account_holder"] == account_holder:
                account["balance"] = balance
        with open(ACCOUNT_FILE, 'w') as file:
            json.dump(accounts, file, indent=2)
    except Exception as e:
        print(f"An unexpected error occurred : {e}")

def access_account():
    name = input("Enter your name: ").strip()
    if name in accounts:
        account = accounts[name]
        while True:
            print("\n------ Account Menu ------")
            print("1. Deposit")
            print("2. Withdraw")
            print("3. Show Details")
            print("4. Transfer to account")
            print("5. Exit")
            choice = input("Enter your choice (1-5): ")
            
            if choice == '1':
                amount = float(input("Enter deposit amount: "))
                account.deposit(amount)
            elif choice == '2':
                amount = float(input("Enter withdrawal amount: "))
                account.withdraw(amount)
            elif choice == '3':
                account.show_details()
            elif choice == '4':
                recipient_name = input("Enter the name of the transfer recipient: ")
                if recipient_name in accounts:
                    recipient = accounts[recipient_name]
                    amount = float(input("Enter transfer amount: "))
                    account.transfer(amount, recipient)
                else:
                    print(f"Recipient account not found. Please verify the name.")
            elif choice == '5':
                print("Exiting account menu.")
                break
            else:
                print("Invalid choice. Please select a valid option.")
    else:
        print("Account not found. Please create an account first.")

# Main menu
while True:
    print("\n----- Bank Account Simulator -----")
    print("1. Create Account")
    print("2. Access Account")
    print("3. Exit")
    choice = input("Enter your choice (1-3):")
    
    if choice == '1':
        create_account()
    elif choice == '2':
        access_account()
    elif choice == '3':
        print("Exiting the program. Goodbye !")
        break
    else:
        print("Invalid choice. Please select a valid option")


----- Bank Account Simulator -----
1. Create Account
2. Access Account
3. Exit



------ Account Menu ------
1. Deposit
2. Withdraw
3. Show Details
4. Transfer to account
5. Exit
Transferred 50.0 € to Indira

------ Account Menu ------
1. Deposit
2. Withdraw
3. Show Details
4. Transfer to account
5. Exit
Withdrew 100.0 €. New balance: 450.0 €

------ Account Menu ------
1. Deposit
2. Withdraw
3. Show Details
4. Transfer to account
5. Exit

-------- Account Details --------
Account Holder: Terry
Account Balance: 450.0 €

------ Account Menu ------
1. Deposit
2. Withdraw
3. Show Details
4. Transfer to account
5. Exit
Exiting account menu.

----- Bank Account Simulator -----
1. Create Account
2. Access Account
3. Exit

------ Account Menu ------
1. Deposit
2. Withdraw
3. Show Details
4. Transfer to account
5. Exit

-------- Account Details --------
Account Holder: Indira
Account Balance: 1300.0 €

------ Account Menu ------
1. Deposit
2. Withdraw
3. Show Details
4. Transfer to account
5. Exit
Deposited 200.0 €. New balance: 1500.0 €

------ Account Menu ------
1. Depo